In [46]:
import json
import pandas as pd
import numpy as np
import typing

from pandas import DataFrame

In [41]:
with open ("Konya_weather.json","r") as f:
    data = json.load(f)
df=pd.json_normalize(data,record_path=['days','hours'],meta=[['days','datetime']])
df


,datetime,datetimeEpoch,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,...,visibility,cloudcover,solarradiation,solarenergy,uvindex,conditions,icon,stations,source,days.datetime
0,00:00:00,1755205200,77.0,77.0,27.57,41.0,0.0,0.0,0.0,0.0,...,6.2,0.0,0.0,0.0,0.0,Clear,clear-night,"[LTAN, 17244099999, TUI0000LTAN]",obs,2025-08-15
1,01:00:00,1755208800,75.2,75.2,27.29,39.2,0.0,0.0,0.0,0.0,...,6.2,0.0,0.0,0.0,0.0,Clear,clear-night,"[LTAN, 17244099999, TUI0000LTAN]",obs,2025-08-15
2,02:00:00,1755212400,77.0,77.0,20.77,33.8,0.0,0.0,0.0,0.0,...,6.2,0.0,0.0,0.0,0.0,Clear,clear-night,"[LTAN, 17244099999, TUI0000LTAN]",obs,2025-08-15
3,03:00:00,1755216000,75.2,75.2,20.52,32.0,0.0,0.0,0.0,0.0,...,6.2,0.0,0.0,0.0,0.0,Clear,clear-night,"[LTAN, 17244099999, TUI0000LTAN]",obs,2025-08-15
4,04:00:00,1755219600,73.4,73.4,21.79,32.0,0.0,0.0,0.0,0.0,...,6.2,0.0,0.0,0.0,0.0,Clear,clear-night,"[LTAN, 17244099999, TUI0000LTAN]",obs,2025-08-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,19:00:00,1763222400,48.1,44.9,53.28,31.9,0.0,0.0,0.0,0.0,...,6.2,88.0,0.0,0.0,0.0,Partially cloudy,partly-cloudy-night,[LTAN],obs,2025-11-15
2228,20:00:00,1763226000,44.5,41.2,56.75,30.1,0.0,0.0,0.0,0.0,...,6.2,50.0,0.0,0.0,0.0,Partially cloudy,partly-cloudy-night,[LTAN],obs,2025-11-15
2229,21:00:00,1763229600,44.5,40.6,56.75,30.1,0.0,0.0,0.0,0.0,...,6.2,50.0,0.0,0.0,0.0,Partially cloudy,partly-cloudy-night,[LTAN],obs,2025-11-15
2230,22:00:00,1763233200,44.5,43.1,56.75,30.1,0.0,0.0,0.0,0.0,...,6.2,1.9,0.0,0.0,0.0,Clear,clear-night,[LTAN],obs,2025-11-15


In [48]:
target_colunms=['days.datetime','datetime','temp','solarradiation','windspeed']
df_final=df[target_colunms].copy()
df_final

,days.datetime,datetime,temp,solarradiation,windspeed
0,2025-08-15,00:00:00,77.0,0.0,5.8
1,2025-08-15,01:00:00,75.2,0.0,6.9
2,2025-08-15,02:00:00,77.0,0.0,4.7
3,2025-08-15,03:00:00,75.2,0.0,1.1
4,2025-08-15,04:00:00,73.4,0.0,8.1
...,...,...,...,...,...
2227,2025-11-15,19:00:00,48.1,0.0,6.9
2228,2025-11-15,20:00:00,44.5,0.0,5.8
2229,2025-11-15,21:00:00,44.5,0.0,6.9
2230,2025-11-15,22:00:00,44.5,0.0,3.4


In [54]:
def convert_units(df1: DataFrame) -> DataFrame:
    #F TO C
    df1['temp_c']=(df1['temp']-32)*5/9
    #MPH TO M/S
    df1['windspeed_ms']=df1['windspeed']*0.44704
    #merging datatime day-hour
    df1['Full Date']=pd.to_datetime(df1['days.datetime']+' - '+df1['datetime'])
    deleted_columns=['days.datetime','datetime','windspeed','temp']
    df1.drop(deleted_columns,inplace=True,axis=1)
    return df1
df_final=convert_units(df_final)

KeyError: 'temp'

In [55]:
df_final

,solarradiation,temp_c,windspeed_ms,Full Date
0,0.0,25.000000,2.592832,2025-08-15 00:00:00
1,0.0,24.000000,3.084576,2025-08-15 01:00:00
2,0.0,25.000000,2.101088,2025-08-15 02:00:00
3,0.0,24.000000,0.491744,2025-08-15 03:00:00
4,0.0,23.000000,3.621024,2025-08-15 04:00:00
...,...,...,...,...
2227,0.0,8.944444,3.084576,2025-11-15 19:00:00
2228,0.0,6.944444,2.592832,2025-11-15 20:00:00
2229,0.0,6.944444,3.084576,2025-11-15 21:00:00
2230,0.0,6.944444,1.519936,2025-11-15 22:00:00
